### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression
import os
print(os.getcwd())
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return str(text).translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

/home/SoliareofAstora/studia1920/analiza_danych/4


,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [36]:
#a)
baby_df['review'] = baby_df["review"].apply(remove_punctuation)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [37]:
#b)
baby_df['review'] = baby_df["review"].apply(lambda x: '' if x=='nan' else x)
#short test:
baby_df["review"][38]

''

In [38]:
#c)
baby_df = baby_df[baby_df["rating"]!=3]
#short test:
sum(baby_df["rating"] == 3)

0

In [39]:
#d) 
baby_df['rating'] = baby_df["rating"].apply(lambda x: -1 if x<3 else 1)
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [41]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [42]:
#a)
baby_df = baby_df.reset_index()

vectors = vectorizer.fit_transform(baby_df.review)
negative = vectors[baby_df.index[baby_df["rating"] == -1]]
positive = vectors[baby_df.index[baby_df["rating"] == 1]]

from scipy.sparse import vstack
X_train = vstack([negative[2600:],
                positive[10000:]])
X_test = vstack([negative[:2600],
                positive[:10000]])

#b)
Y_test = np.array([-1]*2600+[1]*10000)
Y_train = np.array([-1]*(negative.shape[0]-2600)+[1]*(positive.shape[0]-10000))

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [43]:
#a)
model = LogisticRegression()
model.fit(X_train,Y_train)

/home/SoliareofAstora/miniconda3/envs/analiza/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/SoliareofAstora/miniconda3/envs/analiza/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
#b)
weights = model.coef_
ind = np.argpartition(weights[0], 10)[:10]
print('worst',np.array(vectorizer.get_feature_names())[ind])

ind = np.argpartition(weights[0], -10)[-10:]
print('the best',np.array(vectorizer.get_feature_names())[ind])
#hint: model.coef_, vectorizer.get_feature_names()

worst ['dissapointed' 'worthless' 'worst' 'useless' 'theory' 'unusable'
 'disappointing' 'poorly' 'shame' 'poor']
the best ['pleasantly' 'glad' 'sooner' 'ply' 'saves' 'penny' 'skeptical' 'worry'
 'lifesaver' 'downside']


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [45]:
#a)
y_pred = model.predict(X_test)
print(y_pred)

[ 1  1 -1 ...  1  1  1]


In [46]:
#b)
prob = model.predict_proba(X_test)
print(prob)
#hint: model.predict_proba()

[[0.13934721 0.86065279]
 [0.385792   0.614208  ]
 [0.95905345 0.04094655]
 ...
 [0.09259253 0.90740747]
 [0.01792617 0.98207383]
 [0.01782052 0.98217948]]


In [47]:
#c) 
def get_string(index):
    if index<2600:
        return baby_df.review[baby_df["rating"] == -1].iloc[index]
    else:
        return baby_df.review[baby_df["rating"] == 1].iloc[index]

ind = np.argpartition(prob[:,0], -10)[-10:]
for id in ind:
    print('bad ',get_string(id))

print()
ind = np.argpartition(prob[:,1], -10)[-10:]
for id in ind:
    print('good ',get_string(id))

#hint: use the results of b)

bad  Initially I thought these angled bottles make a lot of sense so I put different sizes into our registry However when our baby girl 1 week old first tried to suck the milk out from the bottle it did not come out My husband had activated the vent and did everything according to the instructions He thought the holes were not big enough since he did not see any on the nipple so he put more holes on them but still nothing came out He finally found out the way that these bottles work are not the same as a traditionally bottle does The hole will not open unless pressure is applied against it and hole will get sealed back again when released Baby has to use their tongue to push against the nipples in order to open up the hole For a newborn infant it is too difficult for them to figure out how to get the milk out especially if you are bottle feeding only Luckily we had two Avent bottles with the traditionally hole idea so she is doing very well with the Avent bottles and she still loves th

In [48]:
#d) 
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test,y_pred))

full_model = model
full_test = X_test

0.9107142857142857


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [49]:
#a)
vectorizer = CountVectorizer(vocabulary = 
['love','great','easy','old','little','perfect','loves',
 'well','able','car','broke','less','even','waste',
 'disappointed','work','product','money','would','return'])

vectors = vectorizer.fit_transform(baby_df.review)
negative = vectors[baby_df.index[baby_df["rating"] == -1]]
positive = vectors[baby_df.index[baby_df["rating"] == 1]]

from scipy.sparse import vstack
X_train = vstack([negative[2600:],
                positive[10000:]])
X_test = vstack([negative[:2600],
                positive[:10000]])

Y_test = np.array([-1]*2600+[1]*10000)
Y_train = np.array([-1]*(negative.shape[0]-2600)+[1]*(positive.shape[0]-10000))

model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
#b)

weights = model.coef_
ind = np.argpartition(weights[0], 10)[:10]
print('worst',np.array(vectorizer.get_feature_names())[ind])

ind = np.argpartition(weights[0], -10)[-10:]
print('the best',np.array(vectorizer.get_feature_names())[ind])

y_pred = model.predict(X_test)
# print(y_pred)

prob = model.predict_proba(X_test)
# print(prob)

print("Full dictionary",accuracy_score(Y_test,full_model.predict(full_test)))
print("Limited dictionary",accuracy_score(Y_test,y_pred))

worst ['broke' 'return' 'would' 'money' 'product' 'work' 'disappointed' 'waste'
 'even' 'less']
the best ['car' 'great' 'able' 'well' 'loves' 'perfect' 'little' 'old' 'easy'
 'love']
Full dictionary 0.9107142857142857
Limited dictionary 0.8262698412698413


In [63]:

#c)
%timeit full_prediction = full_model.predict(full_test)

%timeit limited_prediction = model.predict(X_test)
#hint: %time, %timeit

2.69 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
335 µs ± 42.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
